setup

In [20]:
import json, os, re, glob, random, string
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, when, array, concat, size
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array

In [42]:
def write_json(path, data):
    file = open(path, "w+")
    file.write(json.dumps(data)) 
    file.close()

def get_subpaths(path):
    return glob.glob(path)
    
def generate_cat(size):
    return ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(size))

def makedir(dir):
    if not os.path.isdir(dir):
        os.makedirs(dir)

In [3]:
ROOT_DIR = os.path.dirname(os.getcwd()) + "/"
ROOT_DIR

'/Users/user/juhochoi/'

In [4]:
os.environ['SPARK_HOME'] = ROOT_DIR + "resources/spark/"
os.environ['SPARK_HOME']

'/Users/user/juhochoi/resources/spark/'

In [37]:
DATA_DIR = ROOT_DIR + "data/criteo/"
DATA_DIR

'/Users/user/juhochoi/data/criteo/'

In [40]:
TARGET_DIR = DATA_DIR + "train/"
DATA_PATH = DATA_DIR + "full/train.txt"
# TARGET_DIR = DATA_DIR + "sample/"
# DATA_PATH = TARGET_DIR + "sample.txt"
DATA_PATH

'/Users/user/juhochoi/data/criteo/full/train.txt'

In [41]:
SCHEMA_PATH = DATA_DIR + "schema.json"
PART_DIRS = get_subpaths(TARGET_DIR + "part*/")
PART_DIRS

['/Users/user/juhochoi/data/criteo/train/part06/',
 '/Users/user/juhochoi/data/criteo/train/part01/',
 '/Users/user/juhochoi/data/criteo/train/part08/',
 '/Users/user/juhochoi/data/criteo/train/part09/',
 '/Users/user/juhochoi/data/criteo/train/part00/',
 '/Users/user/juhochoi/data/criteo/train/part07/',
 '/Users/user/juhochoi/data/criteo/train/part02/',
 '/Users/user/juhochoi/data/criteo/train/part05/',
 '/Users/user/juhochoi/data/criteo/train/part04/',
 '/Users/user/juhochoi/data/criteo/train/part03/']

In [11]:
spark = SparkSession.builder.appName("pCTR").\
    getOrCreate()

21/10/11 14:01:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/11 14:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [48]:
schema = StructType.fromJson(json.load(open(SCHEMA_PATH)))


In [ ]:
df = spark.read.option("header", "false").option("delimiter", "\t").schema(schema).csv(DATA_PATH)

In [13]:
count = {"total" : df.count()}

In [15]:
label_count = df.groupBy("label").count().orderBy('label').collect()
count["clicked"] = label_count[0]["count"]
count["non_clicked"] = label_count[1]["count"]
count

{'total': 10000, 'clicked': 7818, 'non_clicked': 2182}

In [18]:
PART_DIR = PART_DIRS[0]
PART_DIR

'/Users/user/juhochoi/data/criteo/sample/part00/'

In [44]:
write_json(PART_DIR + "count/count.json", count)

Numeric Features:
- significant numbers of missing values
- skewed distributions 
    - should put median rather than mean for null values
    - for some features with high-frequency of few values, should use mode rather than median.
- large numbers of zeros (described as "mostly count features", so okay)

[article by Alvira Walin](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

## Continuous Features

In [45]:
# Descriptive Statistics
num_features = ["i1", "i2", "i3", "i4", "i5", "i6", "i7", "i8", "i9", "i10", "i11", "i12", "i13"]
num_df = df.select(*num_features)

In [46]:
stats = {}

In [47]:
_, mean, stddev, mn, _, median, _, mx = num_df.summary().collect()

In [48]:
def get_mode(df, c):
    frequency = df.groupBy(c).count().orderBy("count", ascending = False)
    null_count = frequency.filter(df[c].isNull()).collect()
    if null_count:
        null_cnt = null_count[0]["count"]
    else:
        null_cnt = 0
    for row in frequency.head(2):
        if row[c] == None:
            continue
        mode = row[c]
        break
    return int(mode), null_cnt

def key(stat_type, c):
    return "{}({})".format(stat_type, c)

for c in num_df.columns:
    stats[key("min", c)] = int(mn[c])
    stats[key("max", c)] = int(mx[c])
    stats[key("avg", c)] = round(float(mean[c]), 2)
    stats[key("median", c)] = int(median[c])
    stats[key("mode", c)], stats[key("nulls", c)] = get_mode(num_df, c)
    stats[key("stddev", c)] = round(float(stddev[c]), 2)
# stats

In [49]:
write_json(PART_DIR + "stats/stats.json", stats)

## Categorical Features

In [19]:
cat_features = [
        "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", 
        "c11", "c12", "c13", "c14", "c15", "c16", "c17", "c18", "c19", "c20", 
        "c21", "c22", "c23", "c24", "c25", "c26"
    ]
cat_df = df.select(*cat_features)

In [22]:
word_size = 8

In [43]:
def vocabs(dir, df, columns, word_size = 8):
    for c in columns:
        vocab_list = df.select(c).distinct().toPandas()
        while True:
            null_feature = generate_cat(word_size)
            if null_feature not in vocab_list:
                vocab_list = vocab_list.append({c : null_feature}, ignore_index=True)
                break
        vocab_dir = dir + "vocabs/" + c + "/"
        makedir(vocab_dir)
        vocab_list.to_csv(vocab_dir + "vocabs.csv", header = False, index = False)
        count = {"count" : len(vocab_list), "null_feature" : null_feature}
        write_json(vocab_dir + "count.json", count)

In [51]:
for part_dir in PART_DIRS:
    print(part_dir)
    data_path = get_subpaths(part_dir + "raw/*.txt")[0]
    df = spark.read.option("header", "false").option("delimiter", "\t").schema(schema).csv(data_path)
    vocabs(part_dir, df, cat_features)

/Users/user/juhochoi/data/criteo/train/part06/


/Users/user/juhochoi/data/criteo/train/part01/


/Users/user/juhochoi/data/criteo/train/part08/


/Users/user/juhochoi/data/criteo/train/part09/


/Users/user/juhochoi/data/criteo/train/part00/


/Users/user/juhochoi/data/criteo/train/part07/


/Users/user/juhochoi/data/criteo/train/part02/


/Users/user/juhochoi/data/criteo/train/part05/


/Users/user/juhochoi/data/criteo/train/part04/


/Users/user/juhochoi/data/criteo/train/part03/
